In [1]:
!git clone -b atm-10 http://github.com/Zapy67/ATML-PA-2

Cloning into 'ATML-PA-2'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 2), reused 10 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 211.62 KiB | 5.88 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [16]:
!git pull http://github.com/Zapy67/ATML-PA-2 atm-10

From http://github.com/Zapy67/ATML-PA-2
 * branch            atm-10     -> FETCH_HEAD
Already up to date.


In [18]:
!pip install -r requirements.txt

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [ ]:
import os

os.chdir('./ATML-PA-2/DAN')

FileNotFoundError: [Errno 2] No such file or directory: 'ATML-PA-2/DAN'

In [ ]:
import torch
import architecture
from architecture import dan_resnet



<function architecture._forward_impl(self, x)>

In [17]:
from dataset import source_loader, target_loader, ds






ModuleNotFoundError: No module named 'deeplake'